In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def map2R(ss):
    return np.sqrt(np.mean(np.cos(ss), axis=1)**2 + np.mean(np.sin(ss), axis=1)**2)

def theoretical_R(K, w):
    if K >= w:
        return np.cos(0.5 * np.arcsin(w / K))
    else:
        Z = np.pi / np.sqrt(w**2 - K**2)

        x = np.linspace(-np.pi / 2, np.pi / 2, 10000)
        f_x = np.cos(x) / (w - K * np.sin(2 * x))
        return np.sum(f_x * (x[1] - x[0])) / Z


w = 1
K_list = np.linspace(0, 10., 10000)
R_list = np.array([theoretical_R(K, w) for K in K_list])

plt.plot(K_list, R_list)
plt.ylim((-0.01, 1.01))

In [ ]:
beta = 10.0

for R_limit in [0.6, 0.7, 0.9, 0.99]:
    limited_f = np.where(R_list > R_limit, np.exp(- beta * K_list), 0.)
    limited_p = limited_f / (np.sum(limited_f) * (K_list[1] - K_list[0]))

    plt.plot(K_list, np.where(limited_p > 0, limited_p, 0.), label=f"$R^*={R_limit}$")
plt.legend()

In [ ]:
import simulate

def acc_test(K, w, label=None):
    model = simulate.OscilatorNetwork()
    sampling_times = np.linspace(0, 1000, 10000)
    ss = model.solve_ivp(np.array([[0, K], [K, 0]]), w0=np.array([-w, w]), sampling_times=sampling_times)
    R_actuals = map2R(ss)
    
    R_theory = theoretical_R(K, w)
    dR = [np.mean(R_actuals[:n]) - R_theory for n in range(1, ss.shape[0])]
    
    plt.plot(sampling_times[1:], np.abs(dR), label=f"K = {K:.4f}, w = {w:.4f}" if label is None else label)
    

w = 1.0
for K in [0.9, 0.95, 0.99, 1.01, 1.05, 1.1]:
    acc_test(K, w, label=f"K = {K}")

plt.yscale("log")
plt.legend()

In [ ]:
def acc_test_rk4(K, w, label=None, dt=0.01):
    model = simulate.OscilatorNetwork()
    samping_times, ss = model.solve_rk4(np.array([[0, K], [K, 0]]), w0=np.array([-w, w]), T=1000, dt=dt)
    R_actuals = map2R(ss)
    
    R_theory = theoretical_R(K, w)
    dR = [np.mean(R_actuals[:n]) - R_theory for n in range(1, ss.shape[0])]
    
    plt.plot(samping_times[1:], np.abs(dR), label=f"K = {K:.4f}, w = {w:.4f}" if label is None else label)
    

w = 1.0
for K in [0.9, 0.95, 0.99, 1.01, 1.05, 1.1]:
    acc_test_rk4(K, w, label=f"K = {K}")

x = np.linspace(0.01, 1000, 10000)
plt.plot(x, 1 / x, c="black", label="1/t")
plt.yscale("log")
plt.xscale("log")
plt.legend()

In [ ]:
def acc_test_rk4_with_burnin(K, w, label=None, burn_in=100):
    model = simulate.OscilatorNetwork()
    samping_times, ss = model.solve_rk4(np.array([[0, K], [K, 0]]), w0=np.array([-w, w]), T=1000)
    
    ss = ss[samping_times > burn_in]
    samping_times = samping_times[samping_times > burn_in]
    
    R_actuals = map2R(ss)
    
    R_theory = theoretical_R(K, w)
    dR = [np.mean(R_actuals[:n]) - R_theory for n in range(1, ss.shape[0])]
    
    plt.plot(samping_times[1:], np.abs(dR), label=f"K = {K:.4f}, w = {w:.4f}" if label is None else label)
    

w = 1.0
for K in [0.9, 0.95, 0.99, 1.01, 1.05, 1.1]:
    acc_test_rk4_with_burnin(K, w, label=f"K = {K}")

plt.yscale("log")
plt.legend()

In [ ]:
def acc_test_rk4_trapezoid(K, w, label=None):
    model = simulate.OscilatorNetwork()
    samping_times, ss = model.solve_rk4(np.array([[0, K], [K, 0]]), w0=np.array([-w, w]), T=1000)
    R_actuals = map2R(ss)
    
    R_theory = theoretical_R(K, w)
    def trapezoid(t, x):
        n = x.shape[0]
        return ((x[-1] + x[0]) / 2 + np.sum(x[1:-1])) / n
    dR = [trapezoid(samping_times, R_actuals[:n]) - R_theory for n in range(1, ss.shape[0])]
    
    plt.plot(samping_times[1:], np.abs(dR), label=f"K = {K:.4f}, w = {w:.4f}" if label is None else label)
    

w = 1.0
for K in [0.9, 0.95, 0.99, 1.01, 1.05, 1.1]:
    acc_test_rk4_trapezoid(K, w, label=f"K = {K}")

plt.yscale("log")
plt.legend()

In [ ]:
def acc_test_rk4_simpson(K, w, label=None):
    model = simulate.OscilatorNetwork()
    samping_times, ss = model.solve_rk4(np.array([[0, K], [K, 0]]), w0=np.array([-w, w]), T=1000)
    R_actuals = map2R(ss)
    
    R_theory = theoretical_R(K, w)
    def simpson(t, x):
        n = x.shape[0]
        return ((x[0] + x[-1]) + (x[1] + x[-2]) * 5 + np.sum(x[2:-3]) * 6) / (6*n)
    dR = [simpson(samping_times, R_actuals[:n]) - R_theory for n in range(5, ss.shape[0])]
    
    plt.plot(samping_times[5:], np.abs(dR), label=f"K = {K:.4f}, w = {w:.4f}" if label is None else label)
    

w = 1.0
for K in [0.9, 0.95, 0.99, 1.01, 1.05, 1.1]:
    acc_test_rk4_simpson(K, w, label=f"K = {K}")

plt.yscale("log")
plt.legend()

In [ ]:
w = 1.0
K = 0.99
acc_test_rk4(K, w, label=f"square")
acc_test_rk4_trapezoid(K, w, label=f"trapezoid")
acc_test_rk4_simpson(K, w, label=f"simpson")
plt.yscale("log")
plt.legend()

In [ ]:
w = 1.0
K = 0.99
for dt in [0.1, 0.01, 0.001]:
    acc_test_rk4(K, w, label=f"dt = {dt:.4f}", dt=dt)
plt.yscale("log")
plt.legend()